In [1]:
%matplotlib inline

In [2]:
from pathlib import Path
import matplotlib.pyplot as plt

import json
import git
from collections import defaultdict
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
path = Path(".")
if not path.joinpath("cf-action-counter-db").is_dir():
    !git clone https://github.com/regro/cf-action-counter-db

In [4]:
keys = ['appveyor', 'azure-pipelines', 'circleci', 'drone', 'github-actions', 'travis-ci']

In [5]:
totals = defaultdict(dict)

In [6]:
repo = git.Repo('cf-action-counter-db')
ggit = repo.git
for commit in tqdm(repo.iter_commits(rev='master')):
    ggit.checkout(commit)
    if os.path.exists('cf-action-counter-db/data/latest.json'):
        with open('cf-action-counter-db/data/latest.json', 'r') as f:
            data = json.load(f)
        for k in keys:
            if k in data and 'rates' in data[k]:
                totals[k].update(data[k]['rates'])

In [7]:
len(totals['azure-pipelines'])

31022

In [8]:
df = pd.DataFrame(totals)
df.index = pd.to_datetime(df.index)

In [9]:
df.head()

,appveyor,azure-pipelines,circleci,drone,github-actions,travis-ci
2020-05-16 08:50:00-04:00,0.0,4,1.0,0.0,1,1
2020-05-16 08:55:00-04:00,0.0,12,0.0,0.0,4,0
2020-05-16 09:00:00-04:00,0.0,0,0.0,0.0,0,0
2020-05-16 09:05:00-04:00,0.0,6,0.0,0.0,1,0
2020-05-16 09:10:00-04:00,0.0,19,0.0,0.0,5,1


In [10]:
df2.index = pd.to_datetime(df.index, utc=True)

NameError: name 'df2' is not defined

In [ ]:
df2.plot()

In [ ]:
df2.index = pd.DatetimeIndex(df2.index)

In [ ]:
df3 = df2.resample('D').sum()

In [ ]:
df3.head()

In [ ]:
df3.sum()

In [ ]:
for context in [("dark_background", "seaborn-talk"), ("seaborn-talk", )]:
    with plt.style.context(*context):
        for k in df3.columns:
            fig, ax = plt.subplots()
            df3.plot(y=k, ax=ax)
            ax.set_title(f"Number of Builds on {k.title()}")
            ax.set_xlabel("Date (Aggregated by Day)")
            ax.set_ylabel("Number of Builds")
            fig.savefig(f"plots/builds_{k}-{'_'.join(context)}.png", bbox_inches='tight', transparent=True)
    